In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import math
import statistics
import re
import json

In [2]:
business=pd.read_csv("yelp_business.csv")

In [3]:
business= business[business.state == "NC"]
business= business[business.is_open == 1]


In [4]:
def category_splitter(category_string):
    categories = category_string.split(";")   
    return categories
business['categories'] = business['categories'].map(lambda x: category_splitter(x))


In [5]:
def first_list(category_list):
    category_list = category_list[0]
    return category_list
business['categories'] = business['categories'].map(lambda x: first_list(x).lower())
business['categories'] = business['categories'].map(lambda x: re.sub(r'\'', '', x).lower())

business.drop(["business_id","neighborhood","address","review_count","is_open"], axis=1)


,name,city,state,postal_code,latitude,longitude,stars,categories
14,"""Showmars Government Center""",Charlotte,NC,28202,35.221647,-80.839345,3.5,restaurants
42,"""TCBY""",Davidson,NC,28036,35.501227,-80.860949,3.5,desserts
56,"""My Fitness World""",Charlotte,NC,28203,35.220211,-80.855785,5.0,active life
67,"""KS Audio Video""",Cornelius,NC,28031,35.480289,-80.885706,5.0,home services
77,"""Fairfield Inn & Suites""",Charlotte,NC,28105,35.147422,-80.726487,4.0,hotels
84,"""Scott's Karat Patch""",Charlotte,NC,28209,35.160889,-80.849260,4.0,watch repair
100,"""Concord Parkway Animal Hospital""",Concord,NC,28027,35.403844,-80.609093,3.0,pets
102,"""Pep Boys""",Charlotte,NC,28213,35.258682,-80.785106,3.5,auto parts & supplies
103,"""Finga Lickin' Caribbean Eatery""",Charlotte,NC,28205,35.236823,-80.801084,4.5,pizza
126,"""China Buffet""",Charlotte,NC,28262,35.306173,-80.752672,3.5,buffets


In [6]:
business.to_csv("nc_data.csv")

In [7]:
# for typeahead words
raw_unique_cats = list(np.unique(business.categories))

unique_cats = [re.sub(r'\'', '', i).lower() for i in raw_unique_cats]

print(unique_cats)

['acai bowls', 'accessories', 'accountants', 'active life', 'acupuncture', 'adult', 'adult entertainment', 'advertising', 'aerial fitness', 'african', 'air duct cleaning', 'airlines', 'airport shuttles', 'airport terminals', 'airports', 'allergists', 'amateur sports teams', 'american (new)', 'american (traditional)', 'amusement parks', 'antiques', 'apartments', 'appliances', 'appliances & repair', 'arcades', 'archery', 'art classes', 'art galleries', 'art museums', 'art schools', 'art supplies', 'arts & crafts', 'arts & entertainment', 'asian fusion', 'assisted living facilities', 'attraction farms', 'auction houses', 'auto customization', 'auto detailing', 'auto glass services', 'auto insurance', 'auto parts & supplies', 'auto repair', 'automotive', 'baby gear & furniture', 'bagels', 'bakeries', 'bankruptcy law', 'banks & credit unions', 'barbeque', 'barbers', 'barre classes', 'bars', 'battery stores', 'beauty & spas', 'bed & breakfast', 'beer', 'beer bar', 'beer gardens', 'beverage s

# Create JSON of Map Grid of Boxes

### Filter city area out of all businesses

In [8]:
# Charlotte Area
min_long = -81.023171
max_long = -80.653069
min_lat = 35.011523
max_lat = 35.399208

city_biz = business[(business.latitude < max_lat) &
                         (business.latitude > min_lat) &
                         (business.longitude < max_long) &
                         (business.longitude > min_long)]

### Compute lat and long increments

In [9]:
# Compute degrees lat per mile (N-S) and degrees long per mile (E-W)
ns_deg_per_mi = 1/69.01
ew_deg_per_mi = 1/(math.cos(
                            statistics.mean([min_lat, max_lat])*(math.pi/180)
                            ) * 69.1710411)

# Compute miles NS and EW for selected city area
miles_ns = (max_lat - min_lat)/ns_deg_per_mi
miles_ew = (max_long - min_long)/ew_deg_per_mi

delta_long = (max_long - min_long)/miles_ew
delta_lat = (max_lat - min_lat)/miles_ns

### Get counts of number of each category in a grid box

In [10]:
def get_category_counts(minimum_lat, minimum_long, d_lat, d_long):
    category_dict = defaultdict(int)
    for column, biz in business.iterrows():
        if ((biz['latitude'] > minimum_lat) and (biz['latitude'] < minimum_lat + d_lat)) and ((biz['longitude'] > minimum_long) and (biz['longitude'] < minimum_long + d_long)):
            category_dict[biz['categories']] += 1
    if not category_dict:
        category_dict['no business found in this area'] += 1
    return category_dict
        

### Create list of grid box dictionaries

In [11]:
boxes = []
 
i = min_lat
while i < max_lat:
    j = min_long
    while j < max_long:
        cat_counts = get_category_counts(i, j, delta_lat, delta_long)
        box = {'min_lat': i,
              'max_lat': i + delta_lat,
              'min_long': j,
              'max_long': j + delta_long,
              'category_counts': cat_counts}
        print(box)
        boxes.append(box)
        j += delta_long
    i += delta_lat
        

{'min_lat': 35.011523, 'max_lat': 35.02601365352847, 'min_long': -81.023171, 'max_long': -81.00547784401724, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.011523, 'max_lat': 35.02601365352847, 'min_long': -81.00547784401724, 'max_long': -80.98778468803447, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.011523, 'max_lat': 35.02601365352847, 'min_long': -80.98778468803447, 'max_long': -80.9700915320517, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.011523, 'max_lat': 35.02601365352847, 'min_long': -80.9700915320517, 'max_long': -80.95239837606893, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.011523, 'max_lat': 35.02601365352847, 'min_long': -80.95239837606893, 'max_long': -80.93470522008616, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 

{'min_lat': 35.02601365352847, 'max_lat': 35.04050430705695, 'min_long': -80.793159972224, 'max_long': -80.77546681624123, 'category_counts': defaultdict(<class 'int'>, {'arts & entertainment': 1})}
{'min_lat': 35.02601365352847, 'max_lat': 35.04050430705695, 'min_long': -80.77546681624123, 'max_long': -80.75777366025847, 'category_counts': defaultdict(<class 'int'>, {'pet groomers': 1, 'beauty & spas': 1, 'local services': 1, 'chinese': 1})}
{'min_lat': 35.02601365352847, 'max_lat': 35.04050430705695, 'min_long': -80.75777366025847, 'max_long': -80.7400805042757, 'category_counts': defaultdict(<class 'int'>, {'pet services': 1})}
{'min_lat': 35.02601365352847, 'max_lat': 35.04050430705695, 'min_long': -80.7400805042757, 'max_long': -80.72238734829293, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.02601365352847, 'max_lat': 35.04050430705695, 'min_long': -80.72238734829293, 'max_long': -80.70469419231016, 'category_counts': default

{'min_lat': 35.054994960585425, 'max_lat': 35.0694856141139, 'min_long': -81.023171, 'max_long': -81.00547784401724, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.054994960585425, 'max_lat': 35.0694856141139, 'min_long': -81.00547784401724, 'max_long': -80.98778468803447, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.054994960585425, 'max_lat': 35.0694856141139, 'min_long': -80.98778468803447, 'max_long': -80.9700915320517, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.054994960585425, 'max_lat': 35.0694856141139, 'min_long': -80.9700915320517, 'max_long': -80.95239837606893, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.054994960585425, 'max_lat': 35.0694856141139, 'min_long': -80.95239837606893, 'max_long': -80.93470522008616, 'category_counts': defaultdict(<class 'in

{'min_lat': 35.0694856141139, 'max_lat': 35.08397626764238, 'min_long': -80.98778468803447, 'max_long': -80.9700915320517, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.0694856141139, 'max_lat': 35.08397626764238, 'min_long': -80.9700915320517, 'max_long': -80.95239837606893, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.0694856141139, 'max_lat': 35.08397626764238, 'min_long': -80.95239837606893, 'max_long': -80.93470522008616, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.0694856141139, 'max_lat': 35.08397626764238, 'min_long': -80.93470522008616, 'max_long': -80.91701206410339, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.0694856141139, 'max_lat': 35.08397626764238, 'min_long': -80.91701206410339, 'max_long': -80.89931890812062, 'category_counts': defaultdict(<class 

{'min_lat': 35.08397626764238, 'max_lat': 35.09846692117085, 'min_long': -80.98778468803447, 'max_long': -80.9700915320517, 'category_counts': defaultdict(<class 'int'>, {'shopping': 1, 'beauty & spas': 1})}
{'min_lat': 35.08397626764238, 'max_lat': 35.09846692117085, 'min_long': -80.9700915320517, 'max_long': -80.95239837606893, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.08397626764238, 'max_lat': 35.09846692117085, 'min_long': -80.95239837606893, 'max_long': -80.93470522008616, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.08397626764238, 'max_lat': 35.09846692117085, 'min_long': -80.93470522008616, 'max_long': -80.91701206410339, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.08397626764238, 'max_lat': 35.09846692117085, 'min_long': -80.91701206410339, 'max_long': -80.89931890812062, 'category_counts': defaultdict(<cla

...

{'min_lat': 35.30133607056952, 'max_lat': 35.315826724097995, 'min_long': -80.72238734829293, 'max_long': -80.70469419231016, 'category_counts': defaultdict(<class 'int'>, {'bubble tea': 1, 'specialty food': 1, 'family practice': 1, 'auto repair': 1, 'womens clothing': 1, 'car wash': 1, 'pizza': 2, 'beauty & spas': 4, 'hookah bars': 1, 'laundry services': 2, 'bookstores': 1, 'shopping': 2, 'american (traditional)': 1, 'nail salons': 1, 'home services': 2, 'active life': 1, 'university housing': 2, 'event planning & services': 1, 'restaurants': 3, 'coffee & tea': 1, 'gas stations': 1, 'education': 1, 'apartments': 1, 'chinese': 1})}
{'min_lat': 35.30133607056952, 'max_lat': 35.315826724097995, 'min_long': -80.70469419231016, 'max_long': -80.68700103632739, 'category_counts': defaultdict(<class 'int'>, {'grocery': 2, 'hair removal': 2, 'thrift stores': 1, 'pizza': 1, 'juice bars & smoothies': 1, 'pool halls': 1, 'restaurants': 1})}
{'min_lat': 35.30133607056952, 'max_lat': 35.31582672409

{'min_lat': 35.33031737762647, 'max_lat': 35.34480803115495, 'min_long': -81.023171, 'max_long': -81.00547784401724, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.33031737762647, 'max_lat': 35.34480803115495, 'min_long': -81.00547784401724, 'max_long': -80.98778468803447, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.33031737762647, 'max_lat': 35.34480803115495, 'min_long': -80.98778468803447, 'max_long': -80.9700915320517, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.33031737762647, 'max_lat': 35.34480803115495, 'min_long': -80.9700915320517, 'max_long': -80.95239837606893, 'category_counts': defaultdict(<class 'int'>, {'gas stations': 1, 'doctors': 1, 'air duct cleaning': 1, 'child care & day care': 1, 'cosmetic dentists': 1, 'mexican': 1})}
{'min_lat': 35.33031737762647, 'max_lat': 35.34480803115495, 'min_long': -80.952

{'min_lat': 35.34480803115495, 'max_lat': 35.359298684683424, 'min_long': -80.91701206410339, 'max_long': -80.89931890812062, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.34480803115495, 'max_lat': 35.359298684683424, 'min_long': -80.89931890812062, 'max_long': -80.88162575213785, 'category_counts': defaultdict(<class 'int'>, {'flooring': 1, 'pressure washers': 1, 'na': 1, 'trivia hosts': 1, 'restaurants': 1, 'chinese': 1, 'fitness & instruction': 1, 'discount store': 1})}
{'min_lat': 35.34480803115495, 'max_lat': 35.359298684683424, 'min_long': -80.88162575213785, 'max_long': -80.86393259615508, 'category_counts': defaultdict(<class 'int'>, {'sandwiches': 1, 'nail salons': 1, 'chinese': 1, 'drugstores': 1, 'home services': 1})}
{'min_lat': 35.34480803115495, 'max_lat': 35.359298684683424, 'min_long': -80.86393259615508, 'max_long': -80.84623944017231, 'category_counts': defaultdict(<class 'int'>, {'chicken wings': 2, 'beer bar': 

{'min_lat': 35.359298684683424, 'max_lat': 35.3737893382119, 'min_long': -80.77546681624123, 'max_long': -80.75777366025847, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.359298684683424, 'max_lat': 35.3737893382119, 'min_long': -80.75777366025847, 'max_long': -80.7400805042757, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.359298684683424, 'max_lat': 35.3737893382119, 'min_long': -80.7400805042757, 'max_long': -80.72238734829293, 'category_counts': defaultdict(<class 'int'>, {'shopping': 5, 'guitar stores': 1, 'cinema': 1, 'wheel & rim repair': 1, 'restaurants': 4, 'home services': 2, 'burgers': 2, 'salad': 1, 'roof inspectors': 1, 'nail salons': 1, 'apartments': 1, 'churches': 1, 'buffets': 1, 'american (traditional)': 1, 'appliances': 1, 'fast food': 1, 'beauty & spas': 1, 'bubble tea': 1, 'race tracks': 1, 'local services': 1, 'food': 1, 'auto parts & supplies': 1, 'active 

{'min_lat': 35.3737893382119, 'max_lat': 35.388279991740376, 'min_long': -80.68700103632739, 'max_long': -80.66930788034462, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.3737893382119, 'max_lat': 35.388279991740376, 'min_long': -80.66930788034462, 'max_long': -80.65161472436185, 'category_counts': defaultdict(<class 'int'>, {'car rental': 1, 'hotels': 1, 'bakeries': 1})}
{'min_lat': 35.388279991740376, 'max_lat': 35.40277064526885, 'min_long': -81.023171, 'max_long': -81.00547784401724, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.388279991740376, 'max_lat': 35.40277064526885, 'min_long': -81.00547784401724, 'max_long': -80.98778468803447, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 35.388279991740376, 'max_lat': 35.40277064526885, 'min_long': -80.98778468803447, 'max_long': -80.9700915320517, 'category_counts': defaultdic

### Write to file

In [12]:
with open('az_grid_boxes_category_counts.json', 'w') as outfile:
    json.dump(boxes, outfile)